# SAR with Spark and SQL

Re-implementation of SAR reference Python implementation for Spark (distributed) with SQL

In [1]:
# specify parameters
TOP_K=2
RECOMMEND_SEEN=True
# options are 'jaccard', 'lift' or '' to skip and use item cooccurrence directly
SIMILARITY='jaccard'

Sample dataset - feel free to include the large distributed dataset here - these are just here for testing/examples.

In [2]:
# There are two versions of the dataframes - the numeric version and the alphanumeric one:
# they both have similar test data for top-2 recommendations and illustrate the indexing approaches to matrix multiplication on SQL
import pandas as pd
d_train = {
'customerID': [1,1,1,2,2,3,3],
'itemID':     [1,2,3,4,5,6,1],
'rating':     [5,5,5,1,1,3,5]
}
pdf_train = pd.DataFrame(d_train)
d_test = {
'customerID': [1,1,2,2,3,3],
'itemID':     [4,5,1,5,6,1],
'rating':     [1,1,5,5,5,5]
}
pdf_test = pd.DataFrame(d_test)

In [3]:
import numpy as np
a_train = np.array([[5,5,5,0,0,0],\
                    [0,0,0,1,1,0],
                    [5,0,0,0,0,3]])
print(a_train)
print(a_train.shape)

[[5 5 5 0 0 0]
 [0 0 0 1 1 0]
 [5 0 0 0 0 3]]
(3, 6)


In [4]:
d_alnum_train = {
'customerID': ['ua','ua','ua','ub','ub','uc','uc'],
'itemID':     ['ia','ib','ic','id','ie','if','ia'],
'rating':     [5,5,5,1,1,3,5]
}
#pdf_train = pd.DataFrame(d_alnum_train)
pdf_train = pd.DataFrame(d_train)
d_alnum_test = {
'customerID': ['ua','ua','ub','ub','uc','uc'],
'itemID':     ['id','ie','ia','ie','if','ia'],
'rating':     [1,1,5,5,5,5]
}
#pdf_test = pd.DataFrame(d_alnum_test)
pdf_test = pd.DataFrame(d_test)
pdf_test.head(10)

,customerID,itemID,rating
0,1,4,1
1,1,5,1
2,2,1,5
3,2,5,5
4,3,6,5
5,3,1,5


,customerID,itemID,rating
0,ua,ia,5
1,ua,ib,5
2,ua,ic,5
3,ub,id,1
4,ub,ie,1
5,uc,if,3
6,uc,ia,5


In [5]:
from pyspark.sql import SparkSession
spark = SparkSession \
    .builder \
    .appName("Simple") \
    .master("local[*]") \
    .getOrCreate()

In [6]:
import pyspark.sql.functions as F
df = spark.createDataFrame(pdf_train).withColumn("type", F.lit(1))
df_test = spark.createDataFrame(pdf_test).withColumn("type", F.lit(0))
df.show()

+----------+------+------+----+
|customerID|itemID|rating|type|
+----------+------+------+----+
|         1|     1|     5|   1|
|         1|     2|     5|   1|
|         1|     3|     5|   1|
|         2|     4|     1|   1|
|         2|     5|     1|   1|
|         3|     6|     3|   1|
|         3|     1|     5|   1|
+----------+------+------+----+



## Index the user and item IDs

Map user and item alphanumeric IDs to matrix indices.

In [8]:
n_train = df.count()
df_all = df.union(df_test)
df_all.createOrReplaceTempView("df_all")
query = """
SELECT customerid,
       Dense_rank()
         OVER(
           partition BY 1
           ORDER BY customerid) AS row_id,
       itemid,
       Dense_rank()
         OVER(
           partition BY 1
           ORDER BY itemid)     AS col_id,
       rating,
       type
FROM   df_all 
"""
df_all = spark.sql(query)
df_all.createOrReplaceTempView("df_all")
customer_index2ID = dict(df_all.select(["row_id", "customerID"]).rdd.reduceByKey(lambda k, v: v).collect())
item_index2ID = dict(df_all.select(["col_id", "itemID"]).rdd.reduceByKey(lambda k, v: v).collect())
df_all.show()

+----------+------+------+------+------+----+
|customerid|row_id|itemid|col_id|rating|type|
+----------+------+------+------+------+----+
|         1|     1|     1|     1|     5|   1|
|         1|     1|     2|     2|     5|   1|
|         1|     1|     3|     3|     5|   1|
|         1|     1|     4|     4|     1|   0|
|         1|     1|     5|     5|     1|   0|
|         2|     2|     1|     1|     5|   0|
|         2|     2|     4|     4|     1|   1|
|         2|     2|     5|     5|     1|   1|
|         2|     2|     5|     5|     5|   0|
|         3|     3|     1|     1|     5|   1|
|         3|     3|     1|     1|     5|   0|
|         3|     3|     6|     6|     3|   1|
|         3|     3|     6|     6|     5|   0|
+----------+------+------+------+------+----+



## Item Cooccurrence

In [9]:
import pyspark.sql.functions as F
query = """
SELECT row_id,
       col_id,
       rating
FROM   df_all
WHERE  type = 1 
"""
df = spark.sql(query)
df.createOrReplaceTempView("df_train")
df.show()
df_transpose = spark.sql("select col_id as row_id, row_id as col_id, rating from df_train")
df_transpose.createOrReplaceTempView("df_train_transpose")
df_transpose.show()

+------+------+------+
|row_id|col_id|rating|
+------+------+------+
|     1|     1|     5|
|     1|     2|     5|
|     1|     3|     5|
|     2|     4|     1|
|     2|     5|     1|
|     3|     1|     5|
|     3|     6|     3|
+------+------+------+

+------+------+------+
|row_id|col_id|rating|
+------+------+------+
|     1|     1|     5|
|     2|     1|     5|
|     3|     1|     5|
|     4|     2|     1|
|     5|     2|     1|
|     1|     3|     5|
|     6|     3|     3|
+------+------+------+



In [10]:
query = """
SELECT A.row_id AS row_item_id,
       B.col_id AS col_item_id,
       Sum(1)   AS value
FROM   df_train_transpose A
       INNER JOIN df_train B
               ON A.col_id = B.row_id
GROUP  BY A.row_id,
          B.col_id
"""
item_cooccurrence = spark.sql(query)
item_cooccurrence.createOrReplaceTempView("item_cooccurrence")
item_cooccurrence.show()
print(item_cooccurrence.count())

+-----------+-----------+-----+
|row_item_id|col_item_id|value|
+-----------+-----------+-----+
|          6|          1|    1|
|          3|          1|    1|
|          2|          2|    1|
|          2|          3|    1|
|          1|          2|    1|
|          1|          1|    2|
|          1|          3|    1|
|          5|          4|    1|
|          3|          3|    1|
|          2|          1|    1|
|          3|          2|    1|
|          4|          4|    1|
|          6|          6|    1|
|          1|          6|    1|
|          4|          5|    1|
|          5|          5|    1|
+-----------+-----------+-----+

16


In [11]:
indicator = a_train.copy()
indicator[indicator>0]=1
item_cooccurrence = indicator.T.dot(indicator)
print (item_cooccurrence)
print ((item_cooccurrence>0).sum())

[[2 1 1 0 0 1]
 [1 1 1 0 0 0]
 [1 1 1 0 0 0]
 [0 0 0 1 1 0]
 [0 0 0 1 1 0]
 [1 0 0 0 0 1]]
16


In [13]:
pdf_train = pd.DataFrame(d_alnum_train)
pdf_test = pd.DataFrame(d_alnum_test)

df = spark.createDataFrame(pdf_train)
df_test = spark.createDataFrame(pdf_test)
df.show()

+----------+------+------+
|customerID|itemID|rating|
+----------+------+------+
|        ua|    ia|     5|
|        ua|    ib|     5|
|        ua|    ic|     5|
|        ub|    id|     1|
|        ub|    ie|     1|
|        uc|    if|     3|
|        uc|    ia|     5|
+----------+------+------+



In [24]:
query = """
SELECT A.itemID i1, B.itemID i2, count(*) value
FROM   df2 A INNER JOIN df2 B
       ON A.customerID = B.customerID AND 
          A.itemID <= b.itemID  
GROUP  BY A.itemID, B.itemID
"""
df.createOrReplaceTempView("df2")
item_cooccurrence2 = spark.sql(query)
item_cooccurrence2.createOrReplaceTempView("item_cooccurrence2")
item_cooccurrence2.show()

+---+---+-----+
| i1| i2|value|
+---+---+-----+
| ia| if|    1|
| ib| ib|    1|
| ie| ie|    1|
| ia| ib|    1|
| ia| ic|    1|
| if| if|    1|
| ia| ia|    2|
| ib| ic|    1|
| ic| ic|    1|
| id| ie|    1|
| id| id|    1|
+---+---+-----+



In [26]:
item_marginal = spark.sql("SELECT i1 i, value AS margin FROM item_cooccurrence2 WHERE i1 = i2")
item_marginal.createOrReplaceTempView("item_marginal")
item_marginal.show()

+---+------+
|  i|margin|
+---+------+
| ib|     1|
| ie|     1|
| if|     1|
| ia|     2|
| ic|     1|
| id|     1|
+---+------+



In [28]:
query = """
SELECT i1, i2, value / (M1.margin + M2.margin - value) AS value
FROM item_cooccurrence2 A 
    INNER JOIN item_marginal M1 ON A.i1 = M1.i 
    INNER JOIN item_marginal M2 ON A.i2 = M2.i
"""

jaccard2 = spark.sql(query)
jaccard2.createOrReplaceTempView("jaccard2")
jaccard2.show()

+---+---+-----+
| i1| i2|value|
+---+---+-----+
| ic| ic|  1.0|
| ib| ic|  1.0|
| ia| ic|  0.5|
| ib| ib|  1.0|
| ia| ib|  0.5|
| ia| ia|  1.0|
| ia| if|  0.5|
| if| if|  1.0|
| ie| ie|  1.0|
| id| ie|  1.0|
| id| id|  1.0|
+---+---+-----+



In [41]:
query = """
SELECT A1.customerID, S.i2 itemID,
   SUM(A1.rating * S.value) AS score
FROM df2 A1,
     (SELECT J1.i1, J1.i2, J1.value FROM jaccard2 J1 
      UNION ALL 
      SELECT J2.i2 i1, J2.i1 i2, J2.value FROM jaccard2 J2 WHERE J2.i1 <> J2.i2) S
WHERE
 A1.itemID = S.i1
GROUP BY A1.customerID, S.i2 
"""
scores = spark.sql(query)
scores.show()
scores.count()

+----------+------+-----+
|customerID|itemID|score|
+----------+------+-----+
|        uc|    ic|  2.5|
|        uc|    ib|  2.5|
|        ua|    ic| 12.5|
|        ub|    ie|  2.0|
|        ua|    if|  2.5|
|        ua|    ib| 12.5|
|        ub|    id|  2.0|
|        uc|    ia|  6.5|
|        ua|    ia| 10.0|
|        uc|    if|  5.5|
+----------+------+-----+



10

## Item Similarity

In [10]:
# show to who to compute Jaccard
diag = item_cooccurrence.diagonal()
diag_rows = np.expand_dims(diag, axis=0)
diag_cols = np.expand_dims(diag, axis=1)
# this essentially does vstack(diag_rows).T + vstack(diag_rows) - cooccurrence
denom = diag_rows + diag_cols - item_cooccurrence
jaccard = item_cooccurrence / denom
print ("Jaccard")
print (jaccard)

Jaccard
[[1.  0.5 0.5 0.  0.  0.5]
 [0.5 1.  1.  0.  0.  0. ]
 [0.5 1.  1.  0.  0.  0. ]
 [0.  0.  0.  1.  1.  0. ]
 [0.  0.  0.  1.  1.  0. ]
 [0.5 0.  0.  0.  0.  1. ]]


In [11]:
if SIMILARITY is 'jaccard' or SIMILARITY is 'lift':
    query = """
    SELECT A.row_item_id AS i,
           A.value       AS d
    FROM   item_cooccurrence A
    WHERE  A.row_item_id = A.col_item_id 
    """
    diagonal = spark.sql(query)
    diagonal.createOrReplaceTempView("diagonal")

In [12]:
similarity = None
if SIMILARITY is "jaccard":
    query = """
    SELECT A.row_item_id,
           A.col_item_id,
           ( A.value / ( B.d + C.d - A.value ) ) AS value
    FROM   item_cooccurrence AS A,
           diagonal AS B,
           diagonal AS C
    WHERE  A.row_item_id = B.i
           AND A.col_item_id = C.i 
    """
    similarity = spark.sql(query)
elif SIMILARITY is 'lift':
    query = """
    SELECT A.row_item_id,
           A.col_item_id,
           ( A.value / ( B.d * C.d ) ) AS value
    FROM   item_cooccurrence AS A,
           diagonal AS B,
           diagonal AS C
    WHERE  A.row_item_id = B.i
           AND A.col_item_id = C.i 
    """
    similarity = spark.sql(query)
else:
    similarity = item_cooccurrence
similarity.createOrReplaceTempView("item_similarity")
similarity.show()

+-----------+-----------+-----+
|row_item_id|col_item_id|value|
+-----------+-----------+-----+
|          1|          1|  1.0|
|          6|          1|  0.5|
|          3|          1|  0.5|
|          2|          1|  0.5|
|          1|          6|  0.5|
|          6|          6|  1.0|
|          1|          3|  0.5|
|          3|          3|  1.0|
|          2|          3|  1.0|
|          5|          5|  1.0|
|          4|          5|  1.0|
|          5|          4|  1.0|
|          4|          4|  1.0|
|          1|          2|  0.5|
|          3|          2|  1.0|
|          2|          2|  1.0|
+-----------+-----------+-----+



## User Affinity Scores

Multiply User Affinity by the Item Similarity matrix

In [13]:
query = """
SELECT A.row_id                AS row_user_id,
       B.col_item_id,
       Sum(A.rating * B.value) AS score
FROM   df_train A
       INNER JOIN item_similarity B
               ON A.col_id = B.row_item_id
GROUP  BY A.row_id,
          B.col_item_id 
"""
scores = spark.sql(query)
scores.show()
scores.count()


+-----------+-----------+-----+
|row_user_id|col_item_id|score|
+-----------+-----------+-----+
|          3|          1|  6.5|
|          1|          2| 12.5|
|          1|          1| 10.0|
|          1|          3| 12.5|
|          2|          5|  2.0|
|          3|          3|  2.5|
|          2|          4|  2.0|
|          3|          6|  5.5|
|          3|          2|  2.5|
|          1|          6|  2.5|
+-----------+-----------+-----+



10

## Remove Seen Items

Optionally remove items which have already been seen in the training set, i.e. don't recommend items which have been previously bought by the user again.

In [14]:
if not RECOMMEND_SEEN:
    print ("Removing seen items")
    masked_scores = scores\
        .join(df, (scores.row_user_id == df.row_id) & (scores.col_item_id == df.col_id), "left_outer")    
    masked_scores.show()
    # now since training set is smaller, we have nulls under its value column, i.e. item is not in the
    # training set
    masked_scores = \
        masked_scores.withColumn("rating", F.when(F.col('rating').isNull(), F.col('score')).otherwise(0))
else:
    print ("Keeping seen items")
    scores.createOrReplaceTempView("scores")
    masked_scores = spark.sql("select row_user_id, col_item_id, score as rating from scores")
masked_scores.show()

Keeping seen items
+-----------+-----------+------+
|row_user_id|col_item_id|rating|
+-----------+-----------+------+
|          3|          1|   6.5|
|          1|          2|  12.5|
|          1|          1|  10.0|
|          1|          3|  12.5|
|          2|          5|   2.0|
|          3|          3|   2.5|
|          2|          4|   2.0|
|          3|          6|   5.5|
|          3|          2|   2.5|
|          1|          6|   2.5|
+-----------+-----------+------+



## Top-K Item Calculation

In [15]:
from pyspark.sql.window import Window
window = Window.partitionBy(masked_scores["row_user_id"]).orderBy(masked_scores["rating"].desc())
#top_scores =\
#    masked_scores.select("*", F.rank().over(window).alias("top")).filter(F.col("top")<=TOP_K)
top_scores =\
    masked_scores.select("*", F.row_number().over(window).alias("top")).filter(F.col("top")<=TOP_K)
top_scores.show()
top_scores.count()

+-----------+-----------+------+---+
|row_user_id|col_item_id|rating|top|
+-----------+-----------+------+---+
|          1|          2|  12.5|  1|
|          1|          3|  12.5|  2|
|          3|          1|   6.5|  1|
|          3|          6|   5.5|  2|
|          2|          5|   2.0|  1|
|          2|          4|   2.0|  2|
+-----------+-----------+------+---+



6